In [1]:
import sys

In [2]:
sys.path.append("../../../FedLab/")

In [3]:
from fedlab.utils.dataset import CIFAR10Partitioner

In [6]:
x = {'x':2, 'y': 1}
for key in sorted(x.keys()):
    print(key)

x
y
